In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os

2025-04-07 15:49:14.385518: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-07 15:49:15.450692: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-07 15:49:15.450754: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-07 15:49:15.670476: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-07 15:49:15.965296: I tensorflow/core/platform/cpu_feature_guar

In [11]:
# define the encoder and decoder model

# define the necessary time step and input features
input_timesteps = 12
input_features = 32
output_timesteps = 7

# define input
input_layer = tf.keras.layers.Input(shape = (input_timesteps, input_features))

# define the encoder architecture
encoder = tf.keras.layers.LSTM(64, activation = 'relu', return_state = True)
encoder_output, state_h, state_c = encoder(input_layer)

# define the decoder architecture
decoder = tf.keras.layers.LSTM(64, activation = 'relu', return_sequences = True)
decoder_output = decoder(encoder_output, initial_state=[state_h, state_c])

# define dense layer to predict the next 7 time periods
outputs =tf.keras.layers.Dense(output_timesteps)(decoder_output)

# define the model architecture
model = tf.keras.models.Model(input_layer, outputs)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'

In [7]:
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 12, 32)]          0         
                                                                 
 lstm_2 (LSTM)               [(None, 64),              24832     
                              (None, 64),                        
                              (None, 64)]                        
                                                                 
Total params: 24832 (97.00 KB)
Trainable params: 24832 (97.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps):
    # Input layer
    inputs = layers.Input(shape=(input_timesteps, input_features))

    # Encoder (LSTM)
    encoder = layers.LSTM(64, activation='relu', return_state=True)
    encoder_outputs, state_h, state_c = encoder(inputs)

    # Decoder LSTM with its own input sequence (this part does not use encoder outputs as input)
    decoder_inputs = layers.Input(shape=(output_timesteps, input_features))  # Target sequence input
    decoder_lstm = layers.LSTM(64, activation='relu', return_sequences=True)
    
    # Pass encoder states to initialize the decoder LSTM
    decoder_outputs = decoder_lstm(decoder_inputs, initial_state=[state_h, state_c])

    # Dense layer to predict the next 7 time periods (each with 32 features)
    outputs = layers.Dense(input_features)(decoder_outputs)

    # Create the model
    model = models.Model([inputs, decoder_inputs], outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    return model

# Define the input shape and output shape
input_timesteps = 12  # 12 time periods
input_features = 32   # 32 features per time period
output_timesteps = 7  # Predict the next 7 time periods

# Create the model
model = create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps)

# Summarize the model
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_10 (InputLayer)       [(None, 12, 32)]             0         []                            
                                                                                                  
 input_11 (InputLayer)       [(None, 7, 32)]              0         []                            
                                                                                                  
 lstm_15 (LSTM)              [(None, 64),                 24832     ['input_10[0][0]']            
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                            

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps):
    # Input layer
    inputs = layers.Input(shape=(input_timesteps, input_features))

    # Encoder (LSTM)
    encoder = layers.LSTM(64, activation='relu', return_state=True)
    encoder_outputs, state_h, state_c = encoder(inputs)

    # Decoder (LSTM)
    decoder_lstm = layers.LSTM(64, activation='relu', return_sequences=True)
    decoder_outputs = decoder_lstm(encoder_outputs, initial_state=[state_h, state_c])

    # Dense layer to predict the next 7 time periods (each with 32 features)
    outputs = layers.Dense(input_features)(decoder_outputs)

    # Create the model
    model = models.Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    return model

# Define the input shape and output shape
input_timesteps = 12  # 12 time periods
input_features = 32   # 32 features per time period
output_timesteps = 7  # Predict the next 7 time periods

# Create the model
model = create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps)

# Summarize the model
model.summary()


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'NoneType'